<a href="https://colab.research.google.com/github/Sandi-DeLaVega/Tensorflow-Practice/blob/main/tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import warnings
import logging
import tensorflow as tf

In [2]:
#tf.function decorated functions run as static computation graphs
@tf.function
def add(a,b):
  return a + b

@tf.function
def sub(a,b):
  return a - b

@tf.function
def mul(a,b):
  return a*b

@tf.function
def div(a,b):
  return a/b

In [3]:
print(add(tf.constant(5),tf.constant(2)))

tf.Tensor(7, shape=(), dtype=int32)


In [4]:
print(mul(tf.constant(5),tf.constant(2)))

tf.Tensor(10, shape=(), dtype=int32)


In [5]:
print(sub(tf.constant(5),tf.constant(2)))

tf.Tensor(3, shape=(), dtype=int32)


In [6]:
print(div(tf.constant(5),tf.constant(2)))

tf.Tensor(2.5, shape=(), dtype=float64)


In [7]:
@tf.function
def matmul(a,b):
  return tf.matmul(a,b)

In [8]:
@tf.function
def linear(m,x,c):
  return add(matmul(m,x),c)

In [9]:
m = tf.constant([[4.0,5.0,6.0]], tf.float32)

In [10]:
m

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 5., 6.]], dtype=float32)>

In [11]:
x = tf.Variable([[100.0],[100.0],[100.0]],tf.float32)
x

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[100.],
       [100.],
       [100.]], dtype=float32)>

In [12]:
c = tf.constant([[1.0]],tf.float32)
c

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [13]:
linear(m,x,c)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1501.]], dtype=float32)>

In [14]:
@tf.function
def pos_neg_check(x):
  reduce_sum = tf.reduce_sum(x)

  if reduce_sum > 0:
    return tf.constant(1)

  elif reduce_sum == 0:
    return tf.constant(0)

  else:
    return tf.constant(-1)

In [15]:
pos_neg_check(tf.constant([100,100]))

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [16]:
pos_neg_check(tf.constant([100,-100]))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [17]:
pos_neg_check(tf.constant([-100,-100]))

<tf.Tensor: shape=(), dtype=int32, numpy=-1>

In [18]:
num = tf.Variable(7)

In [19]:
@tf.function
def add_times(x):
  for i in tf.range(x):
    num.assign_add(x)

In [20]:
add_times(5)

In [21]:
print(num)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>


In [22]:
a = tf.Variable(1.0)
b = tf.Variable(2.0)

In [23]:
@tf.function
def f(x, y):
  a.assign(y*b)
  b.assign_add(x*a)
  return a + b

In [24]:
f(1,2)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [25]:
@tf.function
def square(a):
  print("Input a: ", a)
  return a*a

Statement with Side Effects
Are executed only the first time when the computation graph is traced

In [26]:
x = tf.Variable([[2,2],[2,2]], dtype =  tf.float32)
square(x)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32>


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 4.],
       [4., 4.]], dtype=float32)>

In [28]:
y = tf.Variable([[2,2],[2,2]], dtype = tf.int32)
square(y)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]], dtype=int32)>

In [31]:
z = tf.Variable([[3,3],[3,3]], dtype = tf.float32)
square(z) #Reuse the Previously traced graph
#Static Computation Graph is Reused to Perform Computation


Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32>


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9., 9.],
       [9., 9.]], dtype=float32)>

#Concrete Function

In [32]:
concrete_int_square_fn = square.get_concrete_function(tf.TensorSpec(shape = None, dtype = tf.int32))
concrete_int_square_fn

Input a:  Tensor("a:0", dtype=int32)


<ConcreteFunction square(a) at 0x7F7AB8232450>

In [33]:
concrete_float_square_fn = square.get_concrete_function(tf.TensorSpec(shape = None, dtype = tf.float32))
concrete_float_square_fn

Input a:  Tensor("a:0", dtype=float32)


<ConcreteFunction square(a) at 0x7F7AB8188050>

In [34]:
concrete_int_square_fn(tf.constant([[2,2],[2,2]], dtype = tf.int32))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]], dtype=int32)>

In [35]:
concrete_float_square_fn(tf.constant([[2.1,2.1],[2.1,2.1]], dtype = tf.float32))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4.4099994, 4.4099994],
       [4.4099994, 4.4099994]], dtype=float32)>

In [36]:
concrete_float_square_fn(tf.constant([[2,2],[2,2]], dtype = tf.float32))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 4.],
       [4., 4.]], dtype=float32)>

In [37]:
@tf.function
def f(x):
  print("Python execution: ", x)
  tf.print("Graph execution: ", x)

In [38]:
f(1)

Python execution:  1
Graph execution:  1


In [39]:
f(1)

Graph execution:  1


In [41]:
def fn_with_variable_init_eager():
  a = tf.constant([[10,10],[11.,1.]])
  x = tf.constant([[1.,0.],[0.,1.]])
  b = tf.Variable(12.)

  y = tf.matmul(a,x) + b

  tf.print("tf_print: ", y)

  return y

In [42]:
@tf.function
def fn_with_variable_init_autograph():
  a = tf.constant([[10,10],[11.,1.]])
  x = tf.constant([[1.,0.],[0.,1.]])
  b = tf.Variable(12.)

  y = tf.matmul(a, x) + b
  tf.print("tf_print: ", y)
  
  return y

eager mode: variable is created each time the function is executed

In [43]:
class F():
  def __init__(self):
    self._b = None

  @tf.function
  def __call__(self):
    a = tf.constant([[10,10],[11.,1]])
    x = tf.constant([[1.,0.],[0.,1.]])

    if self._b is None:
      self._b = tf.Variable(12.)

    y = tf.matmul(a,x) + self._b
    print(y)

    tf.print("tf_print: ", y)
    return y

fn_with_variable_init_autograph = F()
fn_with_variable_init_autograph()

Tensor("add:0", shape=(2, 2), dtype=float32)
Tensor("add:0", shape=(2, 2), dtype=float32)
tf_print:  [[22 22]
 [23 13]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [46]:
def f():
  if x > 0:
    x *= x
  return x

print(tf.autograph.to_code(f))

def tf__f():
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def if_body():
            nonlocal x
            x = ag__.ld(x)
            x *= x

        def else_body():
            nonlocal x
            pass
        x = ag__.Undefined('x')
        ag__.if_stmt((ag__.ld(x) > 0), if_body, else_body, get_state, set_state, ('x',), 1)
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [47]:
#When you execute functions in graph mode
#Tensorflow offers you significant speedups
@tf.function
def g(x):
  return x

start = time.time()
for i in tf.range(2000):
  g(i)
end = time.time()

print("tf.Tensor time elapsed: ", (end - start))

tf.Tensor time elapsed:  0.8549554347991943


In [48]:
#To show how significanly faster
#disable logging
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').disabled = True

In [50]:
start = time.time()
for i in range(2000):
  g(i)
end = time.time()
print("Native type time elapsed: ", (end - start))

Native type time elapsed:  7.787652492523193
